<a href="https://colab.research.google.com/github/Kaustubh-Mundra/3D-Steganography/blob/main/3D_Steganography.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

In [ ]:
import cv2
import numpy as np
import os

In [ ]:
cap = cv2.VideoCapture("input_video.mp4")

frames = []
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frames.append(frame)

cap.release()

volume = np.stack(frames, axis=0)  # (Z, H, W, C)
Z, H, W, C = volume.shape

print("Video volume shape:", volume.shape)

3D Volume Shape: (848, 480, 34, 3)


In [ ]:
key = 5  # simple XOR key (integer)

with open("secret.txt", "r") as f:
    text = f.read()

text_bytes = np.array([ord(c) for c in text], dtype=np.uint8)
encrypted_bytes = text_bytes ^ key

text_length = len(encrypted_bytes)
print("Text length:", text_length)


Embedding Capacity: 10379520 bytes


In [ ]:
length_bits = np.unpackbits(np.array([text_length], dtype=np.uint16))
data_bits = np.unpackbits(encrypted_bytes)

bit_stream = np.concatenate([length_bits, data_bits])


In [ ]:
extension = os.path.splitext(secret_file)[1][1:]
ext_bytes = extension.encode()

file_size_bits = np.unpackbits(np.array([file_size], dtype=np.uint32).view(np.uint8))
ext_len_bits = np.unpackbits(np.array([len(ext_bytes)], dtype=np.uint8))
ext_bits = np.unpackbits(np.frombuffer(ext_bytes, dtype=np.uint8))

embed_stream = np.concatenate(
    [file_size_bits, ext_len_bits, ext_bits, encrypted_bits]
)

if len(embed_stream) > capacity_bits:
    raise ValueError("File too large to embed")


In [ ]:
stego_volume = volume3D.copy()
bit_idx = 0

for z in range(depth):
    for c in range(channels):
        for x in range(rows):
            for y in range(cols):
                if bit_idx >= len(embed_stream):
                    break

                pixel = stego_volume[x, y, z, c]
                # Corrected: Create a uint8-compatible mask to clear the k LSBs
                mask_to_clear_lsbs = 255 - (2**k - 1)
                pixel &= mask_to_clear_lsbs

                bits = embed_stream[bit_idx:bit_idx + k]
                if len(bits) < k:
                    bits = np.pad(bits, (0, k - len(bits)))

                stego_volume[x, y, z, c] = pixel | np.packbits(bits)[0]
                bit_idx += k

In [ ]:
out = cv2.VideoWriter(
    "stego_video.mp4",
    cv2.VideoWriter_fourcc(*"mp4v"),
    30,
    (cols, rows)
)

for i in range(depth):
    out.write(stego_volume[:, :, i, :])

out.release()

print("Stego video created")


Stego video created


Stego Video in Browser

In [ ]:
from IPython.display import HTML
from base64 import b64encode

video_path = "stego_video.mp4"

mp4 = open(video_path, 'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

HTML(f"""
<video width=600 controls>
  <source src="{data_url}" type="video/mp4">
</video>
""")


Download Stego Video

In [ ]:
from google.colab import files
files.download("stego_video.mp4")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

DECONSTRUCTION OF THE ORIGINAL FILE

In [ ]:
extracted_bits = []

for z in range(depth):
    for c in range(channels):
        for x in range(rows):
            for y in range(cols):
                pixel = stego_volume[x, y, z, c]
                bits = np.unpackbits(
                    np.array([pixel & (2**k - 1)], dtype=np.uint8)
                )[-k:]
                extracted_bits.extend(bits)

extracted_bits = np.array(extracted_bits)


In [ ]:
ptr = 0

file_size = np.packbits(extracted_bits[ptr:ptr+32]).view(np.uint32)[0]
ptr += 32

ext_len = np.packbits(extracted_bits[ptr:ptr+8])[0]
ptr += 8

ext = bytes(np.packbits(extracted_bits[ptr:ptr+ext_len*8])).decode()
ptr += ext_len * 8

encrypted_bytes = np.packbits(extracted_bits[ptr:ptr+file_size*8])

# Corrected: Convert the key byte string to a numpy array of uint8 values first
key_bytes_for_decryption = np.frombuffer(key, dtype=np.uint8)
# Then tile this numerical array
key_stream = np.tile(key_bytes_for_decryption, int(np.ceil(file_size / len(key_bytes_for_decryption))))[:file_size]

decrypted_data = np.bitwise_xor(encrypted_bytes, key_stream)

In [ ]:
output_file = f"extracted.{ext}"

with open(output_file, "wb") as f:
    f.write(decrypted_data)

print("Hidden file extracted:", output_file)

Hidden file extracted: extracted.


In [ ]:
from google.colab import files
files.download(output_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>